In [123]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [124]:
total_dict = {}
county_dict = {}
state_dict = {}
dph_illinois_dict = {}
who_world_dict = {}
vul_dict = {}

In [125]:
dynamic_dict = {}
dynamic_county_dict = {}
dynamic_state_dict = {}
dynamic_illinois_dict = {}

1. Read County Data

In [126]:
counties_gdf = gpd.read_file('nyt_counties_data.geojson')

In [127]:
counties_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",366.0,9.0,3.0,0.0,2020-01-21,2020-06-29,day,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",373.0,14.0,5.0,0.0,2020-01-21,2020-06-29,day,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",1920.0,24.0,204.0,2.0,2020-01-21,2020-06-29,day,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,None,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",5.0,0.0,0.0,0.0,2020-01-21,2020-06-29,day,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769,48459,2020-03-20,None,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",46.0,0.0,0.0,0.0,2020-01-21,2020-06-29,day,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [128]:
case = counties_gdf['today_case']
death = counties_gdf['today_death']
case_per_100k_capita = counties_gdf['today_case']/(counties_gdf['population']/100000)
death_per_100k_capita = counties_gdf['today_death']/(counties_gdf['population']/100000)
# death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
# death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [129]:
log_case = case.apply(lambda x: np.log(x + 1))
log_death = death.apply(lambda x: np.log(x + 1))
log_case_per_100k_capita = case_per_100k_capita.apply(lambda x: np.log(x + 1))
log_death_per_100k_capita = death.apply(lambda x: np.log(x + 1))



1.1 Get Constant Classes from today's data

In [130]:
def create_dict(column):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    return tmp_dict

In [131]:
county_dict['case'] = {}
county_dict['death'] = {}
county_dict['case_per_100k_capita'] = {}
county_dict['death_per_100k_capita'] = {}

In [132]:
county_dict['case']['nolog'] = create_dict(case)
county_dict['case']['log'] = create_dict(log_case)
county_dict['death']['nolog'] = create_dict(death)
county_dict['death']['log'] = create_dict(log_death)
county_dict['case_per_100k_capita']['nolog'] = create_dict(case_per_100k_capita)
county_dict['case_per_100k_capita']['log'] = create_dict(log_case_per_100k_capita)
county_dict['death_per_100k_capita']['nolog'] = create_dict(death_per_100k_capita)
county_dict['death_per_100k_capita']['log'] = create_dict(log_death_per_100k_capita)


C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

In [133]:
county_dict

{'case': {'nolog': {'Quantiles': {'bins': '6.0,25.0,68.0,178.0,583.0000000000009,219670.0',
    'k': 6},
   'FisherJenks': {'bins': '2405.0,9203.0,23946.0,45178.0,100772.0,219670.0',
    'k': 6},
   'NaturalBreaks': {'bins': '2233.0,8676.0,23946.0,45178.0,100772.0,219670.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '1.9459101490553132,3.258096538021482,4.23410650459726,5.187385805840755,6.369900982828229,12.29988625148084',
    'k': 6},
   'FisherJenks': {'bins': '1.0986122886681098,2.8903717578961645,4.343805421853684,5.755742213586912,7.512071245835466,12.29988625148084',
    'k': 6},
   'NaturalBreaks': {'bins': '1.0986122886681098,2.8903717578961645,4.31748811353631,5.746203190540153,7.512071245835466,12.29988625148084',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,1.0,4.0,18.0,21941.0', 'k': 6},
   'FisherJenks': {'bins': '232.0,805.0,1595.0,2713.0,4554.0,21941.0', 'k': 6},
   'NaturalBreaks': {'bins': '139.0,503.0,1183.0,2026.0,4554.0,21941.0',
    'k': 6}}

1.2 Create Dynamic classes

In [134]:
# case_ts = counties_gdf['cases_ts'].apply(lambda x: x.split(','))
# death_ts = counties_gdf['deaths_ts'].apply(lambda x: x.split(','))

In [135]:
# length = len(case_ts[1])

In [136]:
# new_case_dict = {}
# for i in range(1,length):
#     q6 = mc.Quantiles(case_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
#     new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

In [137]:
# new_death_dict = {}
# for i in range(1,length):
#     d6 = mc.Quantiles(death_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
#     new_death_dict[i] = ','.join(d6.bins.astype(int).astype(str))

In [138]:
# dynamic_county_dict['new_case'] = new_case_dict
# dynamic_county_dict['new_death'] = new_death_dict


2.Read State Data

In [139]:
state_gdf = gpd.read_file('nyt_states_data.geojson')

In [140]:
state_gdf.head(5)

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2870.0,38.0,93.0,0.0,2020-01-21,2020-06-29,day,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",146333.0,5266.0,3446.0,28.0,2020-01-21,2020-06-29,day,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",143885.0,768.0,7115.0,17.0,2020-01-21,2020-06-29,day,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",35896.0,312.0,1470.0,10.0,2020-01-21,2020-06-29,day,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",67837.0,477.0,3175.0,7.0,2020-01-21,2020-06-29,day,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


In [141]:
state_gdf.iloc[34]

NAME                                                  Virgin Islands
population                                                         0
fips                                                              78
dt_first_case                                             2020-03-14
dt_first_death                                            2020-04-06
cases_ts           0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
deaths_ts          0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
today_case                                                        81
today_new_case                                                     0
today_death                                                        6
today_new_death                                                    0
dt_start                                                  2020-01-21
dt_end                                                    2020-06-29
dt_unit                                                          day
geometry           (POLYGON ((-64.

In [142]:
state_case = state_gdf['today_case']
state_death = state_gdf['today_death']
state_case_per_100k_capita = state_gdf['today_case']/(state_gdf['population']/100000)
state_death_per_100k_capita = state_gdf['today_death']/(state_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [143]:
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.nan,0)
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.inf,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.nan,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.inf,0)

In [144]:
state_log_case = state_case.apply(lambda x: np.log(x + 1))
state_log_death = state_death.apply(lambda x: np.log(x + 1))
state_log_case_per_100k_capita = state_case_per_100k_capita.apply(lambda x: np.log(x + 1))
state_log_death_per_100k_capita =state_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [145]:
state_dict['case'] = {}
state_dict['death'] = {}
state_dict['case_per_100k_capita'] = {}
state_dict['death_per_100k_capita'] = {}

In [146]:
state_dict['case']['nolog'] = create_dict(state_case)
state_dict['case']['log'] = create_dict(state_log_case)
state_dict['death']['nolog'] = create_dict(state_death)
state_dict['death']['log'] = create_dict(state_log_death)
state_dict['case_per_100k_capita']['nolog'] = create_dict(state_case_per_100k_capita)
state_dict['case_per_100k_capita']['log'] = create_dict(state_log_case_per_100k_capita)
state_dict['death_per_100k_capita']['nolog'] = create_dict(state_death_per_100k_capita)
state_dict['death_per_100k_capita']['log'] = create_dict(state_log_death_per_100k_capita)

4. Read DPH Illinois County Data

In [147]:
dph_illinois_gdf = gpd.read_file('illinois/dph_county_data.geojson')

In [148]:
dph_illinois_gdf.head(5)

,id,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,dt_start,dt_end,dt_unit,population,geometry
0,McHenry,McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,1123,40,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,62,5,7054,252,2020-03-17,2020-05-15,day,1,"POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
1,Boone,Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,274,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,14,1,1496,65,2020-03-17,2020-05-15,day,1,"POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
2,Ogle,Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,165,2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,2,0,1677,63,2020-03-17,2020-05-15,day,1,"POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
3,Will,Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,4277,187,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,237,6,24074,1203,2020-03-17,2020-05-15,day,1,"POLYGON ((-88.26146 41.72439, -88.26103 41.708..."
4,LaSalle,LaSalle,"0,0,1,1,1,1,1,3,3,3,3,3,4,5,5,6,6,6,7,8,11,12,...",2020-03-19,117,4,"0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,...",2020-03-29,7,2,2096,78,2020-03-17,2020-05-15,day,1,"POLYGON ((-88.93885 41.62837, -88.93891 41.628..."


In [149]:
dph_illinois_gdf = dph_illinois_gdf[dph_illinois_gdf['id'] != 'Illinois']

In [150]:
dph_illinois_case = dph_illinois_gdf['today_case']

#illinois_case_per_100k_capita = illinois_gdf['today_case']/(illinois_gdf['population']/100000)
#illinois_death_per_100k_capita = illinois_gdf['today_death']/(illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [151]:
# illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
# illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
# illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
# illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [152]:
dph_illinois_log_case = dph_illinois_case.apply(lambda x: np.log(x + 1))
# illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
# illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
# illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [153]:
dph_illinois_dict['case'] = {}
dph_illinois_dict['death'] = {}
dph_illinois_dict['case_per_100k_capita'] = {}
dph_illinois_dict['death_per_100k_capita'] = {}

In [154]:
dph_illinois_dict['case']['nolog'] = create_dict(dph_illinois_case)
dph_illinois_dict['case']['log'] = create_dict(dph_illinois_log_case)
# dph_illinois_dict['death']['nolog'] = create_dict(illinois_death)
# dph_illinois_dict['death']['log'] = create_dict(illinois_log_death)
# dph_illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
# dph_illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
# dph_illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
# dph_illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

6. Read DPH County Test (Static) and DPH Zipcode Case (Static)


In [155]:
dph_illinois_test_gdf = gpd.read_file('illinois/dph_county_static_data.geojson')
dph_zipcode_gdf = gpd.read_file('illinois/dph_zipcode_data.geojson')

In [156]:
dph_zipcode_gdf

,id,confirmed_cases,total_tested,geometry
0,62359,0.0,0.0,"POLYGON ((-91.18290 40.02688, -91.17656 40.020..."
1,62376,0.0,0.0,"POLYGON ((-91.35721 40.17576, -91.35600 40.151..."
2,62312,0.0,0.0,"POLYGON ((-90.97797 39.78881, -90.97787 39.772..."
3,62365,0.0,0.0,"POLYGON ((-91.16584 39.83701, -91.14703 39.837..."
4,62246,6.0,130.0,"POLYGON ((-89.56254 38.91811, -89.56794 38.923..."
...,...,...,...,...
1379,60484,97.0,430.0,"POLYGON ((-87.70969 41.45644, -87.70949 41.456..."
1380,62948,8.0,262.0,"MULTIPOLYGON (((-89.02159 37.74516, -89.03561 ..."
1381,62921,0.0,0.0,"POLYGON ((-89.09155 37.79258, -89.08896 37.807..."
1382,62915,0.0,0.0,"POLYGON ((-89.11994 37.78039, -89.12263 37.781..."


In [157]:
dph_illinois_tested = dph_illinois_test_gdf['total_tested']
dph_zipcode_tested = dph_zipcode_gdf['total_tested']
dph_zipcode_case = dph_zipcode_gdf['confirmed_cases']

In [158]:
dph_illinois_log_tested = dph_illinois_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_tested = dph_zipcode_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_case = dph_zipcode_case.apply(lambda x: np.log(x + 1))

In [159]:
dph_illinois_dict['tested'] = {}
dph_illinois_dict['zipcode_case'] = {}
dph_illinois_dict['zipcode_tested'] = {}

In [160]:
dph_illinois_dict['tested']['nolog'] = create_dict(dph_illinois_tested)
dph_illinois_dict['tested']['log'] = create_dict(dph_illinois_log_tested)
dph_illinois_dict['zipcode_case']['nolog'] = create_dict(dph_zipcode_case)
dph_illinois_dict['zipcode_case']['log'] = create_dict(dph_zipcode_log_case)
dph_illinois_dict['zipcode_tested']['nolog'] = create_dict(dph_zipcode_tested)
dph_illinois_dict['zipcode_tested']['log'] = create_dict(dph_zipcode_log_tested)

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

7. Read Worldwide Data

In [161]:
who_world_gdf = gpd.read_file('worldwide/who_world_data.geojson')

In [162]:
who_world_case = who_world_gdf['today_case']
who_world_death = who_world_gdf['today_death']

In [163]:
who_world_log_case = who_world_case.apply(lambda x: np.log(x + 1))
who_world_log_death = who_world_death.apply(lambda x: np.log(x + 1))

In [164]:
who_world_dict['case'] = {}
who_world_dict['death'] = {}

In [165]:
who_world_dict['case']['nolog'] = create_dict(who_world_case)
who_world_dict['case']['log'] = create_dict(who_world_log_case)
who_world_dict['death']['nolog'] = create_dict(who_world_death)
who_world_dict['death']['log'] = create_dict(who_world_log_death)

8. Read Vulnerability Data

In [166]:
vul_gdf = gpd.read_file('illinois/vulnerability.geojson')

In [167]:
vul = vul_gdf['today_vul']

In [168]:
vul_dict['case'] = {}

In [169]:
vul_dict['case']['nolog'] = create_dict(vul)

Summary and ouput


In [170]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
total_dict['dph_illinois'] = dph_illinois_dict
total_dict['who_world'] = who_world_dict
total_dict['vulnerability'] = vul_dict
# dynamic_dict['county'] = dynamic_county_dict
# dynamic_dict['state'] = dynamic_state_dict
# dynamic_dict['illinois'] = dynamic_illinois_dict
with open('classes.json','w') as json_file:
    json.dump(total_dict, json_file)
# with open('dynamic_classes.json','w') as json_file:
#     json.dump(dynamic_dict, json_file)
print('done')

done
